## alan miranda, cristian orozco, camila rosales


##Suponga un manipulador robotico con 3 grados de libertad. El brazo mide 40cm y el ante brazo mide 50cm. Los angulos de rotación no tienen restricciones. Suponga que la muñeca puede agarrar cualquier objeto que se encuentre a menos de 5cm de ella.

Determine al menos 2 configuraciones de theta1, theta2 y theta3 que permitan alcanzar un objeto en la posición x=45, y=36, z=22. Utilice fuerza bruta para ello o cualquier estrategia objetiva para conseguirlo.

In [ ]:
import numpy as np
from sympy import *

In [ ]:
# Longitudes dadas
l1 = 0
l2 = 40
l3 = 50

# Objetivo
x_obj, y_obj, z_obj = 45, 36, 22
tolerancia = 5  # cm

# Ángulos simbólicos
theta_1, theta_2, theta_3 = symbols('theta_1 theta_2 theta_3')
alpha = symbols('alpha')

Rotación de la base (articulación 1) un angulo theta 1 zobre el eje z

In [ ]:
# Matrices homogéneas simplificadas
_0H1 = Matrix([[cos(theta_1), -sin(theta_1), 0, 0], [sin(theta_1),  cos(theta_1), 0, 0], [0, 0, 1, 0],[0, 0, 0, 1]])
_0H1

Matrix([
[cos(theta_1), -sin(theta_1), 0, 0],
[sin(theta_1),  cos(theta_1), 0, 0],
[           0,             0, 1, 0],
[           0,             0, 0, 1]])

Translación l_1 sobre el eje z desde la base hasta la articulación 2

In [ ]:
_1H2 = Matrix([[1, 0, 0, 0],[0, 1, 0, 0],[0, 0, 1, l1],[0, 0, 0, 1]])
_1H2

Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])

Rotación del sistema de referencia (90 grados sobre el eje X)

In [ ]:
_2H3 = Matrix([[1, 0, 0, 0], [0, cos(alpha), -sin(alpha), 0], [0, sin(alpha), cos(alpha), 0], [0, 0, 0, 1]])
_2H3

Matrix([
[1,          0,           0, 0],
[0, cos(alpha), -sin(alpha), 0],
[0, sin(alpha),  cos(alpha), 0],
[0,          0,           0, 1]])

In [ ]:
_2H3 = _2H3.subs(alpha, pi/2)
_2H3

Matrix([
[1, 0,  0, 0],
[0, 0, -1, 0],
[0, 1,  0, 0],
[0, 0,  0, 1]])

Rotación theta_2 sobre el nuevo eje Z en la articulación 2

In [ ]:
_3H4 = Matrix([[cos(theta_2), -sin(theta_2), 0, 0],[sin(theta_2),  cos(theta_2), 0, 0],[0, 0, 1, 0],[0, 0, 0, 1]])
_3H4

Matrix([
[cos(theta_2), -sin(theta_2), 0, 0],
[sin(theta_2),  cos(theta_2), 0, 0],
[           0,             0, 1, 0],
[           0,             0, 0, 1]])

Translación l_2 sobre el nuevo eje X desde la articulación 2 hasta la articulación 3

In [ ]:

_4H5 = Matrix([[1, 0, 0, l2],[0, 1, 0, 0],[0, 0, 1, 0],[0, 0, 0, 1]])
_4H5

Matrix([
[1, 0, 0, 40],
[0, 1, 0,  0],
[0, 0, 1,  0],
[0, 0, 0,  1]])

Rotación theta_3 sobre el nuevo eje Z para la articulación 3

In [ ]:
_5H6 = Matrix([[cos(theta_3), -sin(theta_3), 0, 0],[sin(theta_3),  cos(theta_3), 0, 0],[0, 0, 1, 0],[0, 0, 0, 1]])



_5H6

Matrix([
[cos(theta_3), -sin(theta_3), 0, 0],
[sin(theta_3),  cos(theta_3), 0, 0],
[           0,             0, 1, 0],
[           0,             0, 0, 1]])

Traslación l_3 desde la articulación 3 hasta el efector final (muñeca)

In [ ]:
_6H7 = Matrix([[1, 0, 0, l3],[0, 1, 0, 0],[0, 0, 1, 0],[0, 0, 0, 1]])
_6H7

Matrix([
[1, 0, 0, 50],
[0, 1, 0,  0],
[0, 0, 1,  0],
[0, 0, 0,  1]])

In [ ]:

# Composición
H = simplify(_0H1 @ _1H2 @ _2H3 @ _3H4 @ _4H5 @ _5H6 @ _6H7)


Esta será la posición final de la muñeca en función de theta1, theta2, theta3, l1, l2 y l3

In [ ]:
# posición del efector final
P = H[0:3, 3]

In [ ]:


# Fuerza bruta
encontrados = []
for t1 in np.deg2rad(np.arange(0, 360, 100)): #rangos de rotacion del brazo y nuero de iteraciones
    for t2 in np.deg2rad(np.arange(0, 360, 100)): #rangos de rotacion del brazo y nuero de iteraciones
        for t3 in np.deg2rad(np.arange(0, 360, 100)): #rangos de rotacion del brazo y nuero de iteraciones
            Px = P[0].subs({theta_1: t1, theta_2: t2, theta_3: t3, alpha: alpha, l1: l1, l2: l2, l3: l3})
            Py = P[1].subs({theta_1: t1, theta_2: t2, theta_3: t3, alpha: alpha, l1: l1, l2: l2, l3: l3})
            Pz = P[2].subs({theta_1: t1, theta_2: t2, theta_3: t3, alpha: alpha, l1: l1, l2: l2, l3: l3})

            Px, Py, Pz = float(Px), float(Py), float(Pz)
            dist = np.sqrt((x_obj - Px)**2 + (y_obj - Py)**2 + (z_obj - Pz)**2)

            if dist <= tolerancia:
                encontrados.append((np.rad2deg(t1), np.rad2deg(t2), np.rad2deg(t3), dist))
                if len(encontrados) >= 2:
                    break
        if len(encontrados) >= 2:
            break
    if len(encontrados) >= 2:
        break

# Resultados
for i, (t1, t2, t3, d) in enumerate(encontrados, 1):
    print(f"Configuración {i}: θ1={t1:.2f}°, θ2={t2:.2f}°, θ3={t3:.2f}°, distancia al objetivo = {d:.2f} cm")

Configuración 1: θ1=40.00°, θ2=70.00°, θ3=270.00°, distancia al objetivo = 3.66 cm
Configuración 2: θ1=40.00°, θ2=80.00°, θ3=260.00°, distancia al objetivo = 3.93 cm


respecto ala pregunta de cual era la funcion de la matriz  _6H7 se entiende que  es solo una traslación de l3 en X sin rotaciones.